## Labelling data for multi-label classification

In [1]:
from dotenv import load_dotenv
import json
import time
from typing import Tuple, Set, List
import pandas as pd
from tqdm import tqdm
import json
import re
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
import os

In [2]:
df = pd.read_csv("./data/cleaned_transcripts.csv")

In [3]:
df.head()

,file_name,episode_transcript,length,tags,creator,cleaned_transcript
0,أطفال_غزة,بعد سبعه اشهر وانا لا اجرؤ على حذف شيء\nيخصك م...,00:00:35,"['غزة', 'الدحيح', 'AJ+ عربي', 'Eldahih السليط'...",Al Mokhbir Al Eqtisadi,سبعه اشهر وانا اجرؤ حذف شيء يخصك ازيل تطبيقات ...
1,المخبر_الاقتصادي___كيف_أصبح_أصدقاء_إسرائيل_من_...,عمان وقطر والبحرين والسعوديه دي هي الدول\nالعر...,00:22:52,"['اسرائيل', 'اشرف ابراهيم', 'البرلمان البريطان...",Al Mokhbir Al Eqtisadi,عمان وقطر والبحرين والسعوديه الدول العربيه الا...
2,المخبر_الاقتصادي___كيف_بنى_المهاجرون_المغاربيو...,على غير العادة وبدون أي مقدمات\nخلونا نبدأ الح...,00:20:44,"['الجزائر', 'المغرب العربي', 'المغرب الكبير', ...",Al Mokhbir Al Eqtisadi,العادة وبدون مقدمات خلونا نبدأ الحلقة وندخل ال...
3,المخبر_الاقتصادي___كيف_تخطط_أمريكا_لمنع_تطور_ق...,من حوالي السنتين\nوتحديداً في 3 ديسمبر 2020\nج...,00:14:52,"['AMD', 'APPLE', 'CHINA', 'NVDA', 'QCOM', 'SEC...",Al Mokhbir Al Eqtisadi,السنتين وتحديدا جون راتكليف مدير الاستخبارات ا...
4,المخبر_الاقتصادي___كيف_سرقت_بريطانيا_كوه_نور_أ...,يوم 8 سبتمبر 2022\nتم إعلان وفاة ملكة بريطانيا...,00:14:54,"['الملكة', 'الملكة إليزابيث', 'المملكة المتحدة...",Al Mokhbir Al Eqtisadi,إعلان وفاة ملكة بريطانيا إليزبيث عمر يناهز الش...


### Gemini

In [ ]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

def extract_json_object(s: str) -> dict:
    """
    Extracts a JSON object from a given string.
    
    If the string is wrapped in markdown code fences (e.g., ```json ... ```),
    this function extracts only the JSON content. Otherwise, it assumes the string
    itself is a valid JSON representation.
    
    Args:
        s (str): The input string containing a JSON object.
    
    Returns:
        dict: The parsed JSON object.
    
    Raises:
        ValueError: If no valid JSON object can be parsed from the string.
    """
    # Try to extract content inside markdown code fences
    pattern = re.compile(r"```(?:json)?\s*(\{.*\})\s*```", re.DOTALL)
    match = pattern.search(s)
    if match:
        json_str = match.group(1)
    else:
        json_str = s.strip()
    
    try:
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        raise ValueError(f"Error parsing JSON: {e}") from e

def classify_arabic_transcripts_with_history(
    transcripts: List[str], 
    llm: ChatGoogleGenerativeAI, 
    previous_labels: Set[str] = None
) -> Tuple[str, Set[str]]:
    """
    Classifies a batch of Arabic transcripts (multiple transcripts in one API call)
    and returns a JSON string mapping each transcript to its labels, along with an
    updated set of labels (including previous ones).

    Args:
        transcripts: A list of Arabic transcript strings.
        llm: A LangChain model instance for classification.
        previous_labels: A set of previously identified Arabic labels (optional).

    Returns:
        A tuple:
            - A JSON string containing the classifications for each transcript.
            - An updated set of all unique Arabic labels encountered.
    """
    # Combine all transcripts into one string with numbering.
    transcripts_text = "\n\n".join(
        [f"Transcript {i+1}: {transcript}" for i, transcript in enumerate(transcripts)]
    )
    
    previous_labels_prompt = ""
    if previous_labels:
        previous_labels_prompt = (
            f"Keep in mind these previously mentioned labels: {', '.join(previous_labels)}. "
            "Use them if applicable, and add new labels if needed.\n"
        )
    
    prompt_template = ChatPromptTemplate.from_template(
        """
        You are an expert in Arabic text classification.
        For the following Arabic transcripts, provide for each transcript a list of relevant, concise labels in arabic 
        that describe its main topics or themes. Each label should be a short noun or noun phrase.
        Output your answer as a JSON object where the keys are "transcript_1", "transcript_2", etc.,
        and the values are lists of labels.
        Do not include any additional text, commentary, or explanations.

        {previous_labels_prompt}

        Transcripts:
        {transcripts_text}

        JSON Output:
        """
    )
    
    output_parser = StrOutputParser()
    chain = prompt_template | llm | output_parser

    # Invoke the chain once with the batch of transcripts.
    json_output = chain.invoke({
        "previous_labels_prompt": previous_labels_prompt,
        "transcripts_text": transcripts_text
    })
    
    try:
        batch_dict = extract_json_object(json_output)
    except Exception as e:
        print(f"Error parsing JSON output: {e}")
        batch_dict = {}
    
    all_labels = set(previous_labels) if previous_labels else set()
    for key, labels in batch_dict.items():
        for label in labels:
            all_labels.add(label.strip())
    
    return json.dumps(batch_dict, indent=2, ensure_ascii=False), all_labels 


def process_batches(df: pd.DataFrame, column: str, n: int = 5) -> Tuple[pd.DataFrame, Set[str]]:
    """
    Processes transcripts in batches of n and classifies them using a single LLM call per batch.
    The function adds a new column "classifications" to the DataFrame, where each row is updated 
    with a list of labels (obtained from the LLM) corresponding to the transcript.
    It also updates a global set of unique labels across all batches.

    Args:
        df: A DataFrame containing the transcripts.
        column: The column name containing the transcripts.
        n: The batch size for processing.

    Returns:
        A tuple containing:
            - The updated DataFrame with a new "classifications" column.
            - A set of all unique labels encountered.
    """
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0, api_key=GOOGLE_API_KEY)
    all_labels = set()

    if "classifications" not in df.columns:
        df["classifications"] = None

    llm_request_counter = 0

    for i in tqdm(range(0, len(df), n), desc="Processing batches"):
        transcripts_batch = df[column].iloc[i:i+n].tolist()

        json_output, batch_labels = classify_arabic_transcripts_with_history(transcripts_batch, llm, all_labels)
        all_labels.update(batch_labels)
        llm_request_counter += 1

        try:
            batch_dict = json.loads(json_output)
            print("Batch dict:", batch_dict)
        except Exception as e:
            print(f"Error parsing JSON output for batch starting at index {i}: {e}")
            batch_dict = {}
        
        # Assign each transcript's labels from the JSON to the corresponding DataFrame row.
        # Keys are assumed to be "transcript_1", "transcript_2", etc.
        for j, key in enumerate(sorted(batch_dict.keys(), key=lambda x: int(x.split('_')[-1]))):
            df_index = i + j
            if df_index < len(df):
                df.at[df.index[df_index], "classifications"] = batch_dict[key]
        
        # Sleep for 20 seconds after every 15 LLM requests to avoid rate limits.
        if llm_request_counter % 15 == 0:
            print("Reached 15 requests. Sleeping for 20 seconds to avoid rate limits...")
            time.sleep(20)

        print(f"Processed batch ending at transcript index {i+n}.")
        print("----------------------------------------------")
    
    return df, all_labels


In [ ]:
# # Example usage (with Arabic transcripts):
# llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0, api_key=GOOGLE_API_KEY)
# transcripts_batch1 = [
#     "يتحدث هذا الفيديو عن تاريخ استكشاف الفضاء وأول هبوط على سطح القمر.",
#     "تركز الحلقة على تقنيات الطبخ لخبز خبز العجين المخمر.",
#     "فيلم وثائقي عن تأثير تغير المناخ على القمم الجليدية القطبية.",
#     "درس تعليمي حول كيفية عزف أغنية شعبية على الجيتار.",
#     "حديث عن الذكاء الاصطناعي وتطبيقاته المستقبلية المحتملة."
# ]

# json_output1, all_labels1 = classify_arabic_transcripts_with_history(transcripts_batch1, llm)
# print(json_output1)
# print(f"جميع التصنيفات: {all_labels1}")
# print("-" * 20)

# transcripts_batch2 = [
#     "هذا الفيديو يتحدث عن فوائد التأمل واليقظة الذهنية.",
#     "مراجعة لأحدث سيارة كهربائية وميزاتها.",
#     "مناقشة حول التأثير الاقتصادي للعولمة.",
#     "دليل لزراعة ورعاية حديقة الخضروات.",
#     "غوص عميق في فيزياء ميكانيكا الكم."
# ]

# json_output2, all_labels2 = classify_arabic_transcripts_with_history(transcripts_batch2, llm, all_labels1)
# print(json_output2)
# print(f"جميع التصنيفات: {all_labels2}")
# print("-" * 20)

# transcripts_batch3 = [
#     "فيديو يشرح عملية التمثيل الضوئي في النباتات.",
#     "هذا الفيديو عن تاريخ مصر القديمة وفراعنتها.",
#     "درس تعليمي حول كيفية بناء موقع ويب بسيط باستخدام HTML و CSS.",
#     "مناقشة حول الاعتبارات الأخلاقية لتحرير الجينات.",
#     "مراجعة لفيلم خيال علمي شهير."
# ]

# json_output3, all_labels3 = classify_arabic_transcripts_with_history(transcripts_batch3, llm, all_labels2)
# print(json_output3)
# print(f"جميع التصنيفات: {all_labels3}")

In [ ]:
df, all_labels = process_batches(df, 'cleaned_transcript', n=5)

Processing batches:   1%|          | 1/86 [00:04<06:15,  4.41s/it]

Batch dict: {'transcript_1': ['امهات غزه', 'اطفال غزه', 'قتل الاطفال', 'رفح', 'الاحتلال'], 'transcript_2': ['العلاقات البريطانية الإسرائيلية', 'اللوبي الإسرائيلي في بريطانيا', 'دعم بريطانيا لإسرائيل', 'حزب المحافظين', 'غزة', 'السياسة البريطانية في الشرق الأوسط', 'العدوان على غزة', 'الاحتلال الإسرائيلي', 'الكونسير فرينز اوف اسرائيل', 'اصدقاء اسرائيل حزب المحافظين'], 'transcript_3': ['الهجرة المغاربية إلى فرنسا', 'الاستعمار الفرنسي للمغرب العربي', 'أوضاع العمال المغاربة في فرنسا', 'التمييز ضد المغاربة في فرنسا', 'المنطقة المغاربية', 'الجزائر', 'المغرب', 'تونس'], 'transcript_4': ['الحرب التكنولوجية بين أمريكا والصين', 'صناعة أشباه الموصلات', 'الذكاء الاصطناعي', 'العقوبات الأمريكية على الصين', 'التحالف التكنولوجي الرباعي', 'شركة هواوي', 'شركة انفيديا', 'كوريا الجنوبية', 'اليابان', 'تايوان'], 'transcript_5': ['جوهرة كوه نور', 'الاستعمار البريطاني للهند', 'شركة الهند الشرقية', 'المطالبات باستعادة كوه نور', 'تاريخ كوه نور', 'التاج الملكي البريطاني', 'الملكة إليزابيث', 'الماس']}
Processed batc

Processing batches:   2%|▏         | 2/86 [00:07<05:29,  3.92s/it]

Batch dict: {'transcript_1': ['صندوق الاسكا الدائم', 'ايرادات النفط والغاز', 'توزيع ارباح النفط', 'ولاية الاسكا الامريكية', 'هوجو تشافيز', 'الصناديق السيادية'], 'transcript_2': ['سوق النفط', 'أوبك', 'النفط الروسي', 'أسعار النفط', 'الاستقلال في مجال الطاقة', 'المنتجين الأمريكيين للنفط', 'التحالف الغربي', 'تخفيض انتاج النفط'], 'transcript_3': ['ازمة الشحن العالمية', 'قناة بنما', 'البحر الاحمر', 'هجمات الحوثيين', 'قناة السويس', 'اسعار الشحن', 'التجارة العالمية', 'الموانئ الصينية', 'راس الرجاء الصالح'], 'transcript_4': ['معدلات الخصوبة', 'كوريا الجنوبية', 'التركيبة السكانية', 'الزواج', 'الاحلال السكاني', 'السياسات الاجتماعية', 'الاقتصاد الكوري الجنوبي'], 'transcript_5': ['العقوبات الأمريكية على الصين', 'الحرب التكنولوجية بين أمريكا والصين', 'العناصر الأرضية النادرة', 'الاعتماد الاقتصادي على الصين', 'الحرب التجارية', 'تايوان', 'العلاقات الصينية الأمريكية']}
Processed batch ending at transcript index 10.
----------------------------------------------


Processing batches:   3%|▎         | 3/86 [00:11<05:17,  3.83s/it]

...

Error parsing JSON output: Error parsing JSON: Expecting value: line 1 column 1 (char 0)
Batch dict: {}
Processed batch ending at transcript index 420.
----------------------------------------------


Processing batches:  99%|█████████▉| 85/86 [21:46<00:24, 24.36s/it]

Batch dict: {'transcript_1': ['مكيافيلي', 'كتاب الأمير', 'السياسة', 'الأخلاق', 'السلطة', 'الاستبداد', 'الديمقراطية', 'الفلسفة السياسية', 'الجمهورية', 'الخيانة', 'الخوف', 'الخداع', 'الغاية تبرر الوسيلة', 'الفساد', 'الظلم', 'الرغبة', 'العدالة', 'الخلافة', 'الخلافات', 'الخلاف', 'الخيانة', 'الخوف', 'الخداع', 'الغاية تبرر الوسيلة', 'الفساد', 'الظلم', 'الرغبة', 'العدالة', 'الخلافة', 'الخلافات', 'الخلاف'], 'transcript_2': ['التطور', 'نظرية التطور', 'الحفريات', 'القرآن', 'التفسير', 'العلوم', 'الدين', 'الاستعمار', 'العنصرية', 'الداروينية الاجتماعية', 'الرؤية الدينية', 'الخرافات', 'الالحاد', 'الجهل', 'التعصب', 'التحيز', 'التحيز الإعلامي', 'التحيز', 'التحيز الإعلامي'], 'transcript_3': ['القضية الفلسطينية', 'غزة', 'الاحتلال الإسرائيلي', 'الإبادة الجماعية', 'الفصل العنصري', 'المقاومة', 'الأسلحة', 'التكنولوجيا العسكرية', 'الاستعمار', 'الإمبريالية', 'الفاشية', 'العنصرية', 'حقوق الإنسان', 'المظاهرات', 'كتائب عز الدين القسام', 'الجيش الإسرائيلي', 'الأسلحة الإسرائيلية', 'الشركات الإسرائيلية', 'السياسة ا

Processing batches: 100%|██████████| 86/86 [22:22<00:00, 15.61s/it]

Error parsing JSON output: Error parsing JSON: Expecting value: line 1 column 1 (char 0)
Batch dict: {}
Processed batch ending at transcript index 430.
----------------------------------------------


In [16]:
df.to_csv("./data/gemini_multi_label_classifications.csv", index=False)

In [ ]:
import pickle
with open('./data/all_labels.pkl', 'wb') as f:
    pickle.dump(all_labels, f)

In [17]:
df = pd.read_csv("./data/gemini_multi_label_classifications.csv")

In [18]:
df.head()

,file_name,episode_transcript,length,tags,creator,cleaned_transcript,classifications
0,أطفال_غزة,بعد سبعه اشهر وانا لا اجرؤ على حذف شيء\r\nيخصك...,00:00:35,"['غزة', 'الدحيح', 'AJ+ عربي', 'Eldahih السليط'...",Al Mokhbir Al Eqtisadi,سبعه اشهر وانا اجرؤ حذف شيء يخصك ازيل تطبيقات ...,"['امهات غزه', 'اطفال غزه', 'قتل الاطفال', 'رفح..."
1,المخبر_الاقتصادي___كيف_أصبح_أصدقاء_إسرائيل_من_...,عمان وقطر والبحرين والسعوديه دي هي الدول\r\nال...,00:22:52,"['اسرائيل', 'اشرف ابراهيم', 'البرلمان البريطان...",Al Mokhbir Al Eqtisadi,عمان وقطر والبحرين والسعوديه الدول العربيه الا...,"['العلاقات البريطانية الإسرائيلية', 'اللوبي ال..."
2,المخبر_الاقتصادي___كيف_بنى_المهاجرون_المغاربيو...,على غير العادة وبدون أي مقدمات\r\nخلونا نبدأ ا...,00:20:44,"['الجزائر', 'المغرب العربي', 'المغرب الكبير', ...",Al Mokhbir Al Eqtisadi,العادة وبدون مقدمات خلونا نبدأ الحلقة وندخل ال...,"['الهجرة المغاربية إلى فرنسا', 'الاستعمار الفر..."
3,المخبر_الاقتصادي___كيف_تخطط_أمريكا_لمنع_تطور_ق...,من حوالي السنتين\r\nوتحديداً في 3 ديسمبر 2020\...,00:14:52,"['AMD', 'APPLE', 'CHINA', 'NVDA', 'QCOM', 'SEC...",Al Mokhbir Al Eqtisadi,حوالي السنتين وتحديدا جون راتكليف مدير الاستخب...,"['الحرب التكنولوجية بين أمريكا والصين', 'صناعة..."
4,المخبر_الاقتصادي___كيف_سرقت_بريطانيا_كوه_نور_أ...,يوم 8 سبتمبر 2022\r\nتم إعلان وفاة ملكة بريطان...,00:14:54,"['الملكة', 'الملكة إليزابيث', 'المملكة المتحدة...",Al Mokhbir Al Eqtisadi,إعلان وفاة ملكة بريطانيا إليزبيث عمر يناهز الش...,"['جوهرة كوه نور', 'الاستعمار البريطاني للهند',..."


### TF-IDF labelling

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer() 
tfidf_matrix = vectorizer.fit_transform(df["cleaned_transcript"])
print(tfidf_matrix.shape)

feature_names = vectorizer.get_feature_names_out()
print(feature_names[:100])

def get_top_tfidf_words(row):
    row_data = row.toarray().flatten()  # Convert sparse row to array
    top_indices = row_data.argsort()[-3:][::-1]  # Get indices of top 3 scores
    top_words = [feature_names[i] for i in top_indices]  # Get corresponding words
    return ', '.join(top_words)



(426, 108547)
['__' 'ìū' 'ōé' 'آاااااه' 'آبا' 'آباء' 'آباؤهم' 'آباد' 'آبار' 'آبل' 'آبي'
 'آت' 'آتون' 'آتيك' 'آثار' 'آثاره' 'آثارها' 'آثامهم' 'آجا' 'آجل' 'آجلا'
 'آجي' 'آجيب' 'آجيت' 'آخد' 'آخدش' 'آخدك' 'آخدلها' 'آخده' 'آخدها' 'آخدهم'
 'آخرا' 'آخرة' 'آخرته' 'آخرتها' 'آخرك' 'آخرنا' 'آخره' 'آخرها' 'آخرهم'
 'آخري' 'آخرين' 'آد' 'آداء' 'آدائك' 'آداب' 'آدم' 'آدمي' 'آدمية' 'آدمين'
 'آدي' 'آديك' 'آذان' 'آر' 'آراء' 'آراءكم' 'آرائهم' 'آرت' 'آرتر' 'آرثر'
 'آردي' 'آرسيلور' 'آرش' 'آرشة' 'آرليت' 'آرليس' 'آرم' 'آرنت' 'آرون' 'آري'
 'آريس' 'آرييل' 'آزوفستال' 'آزيم' 'آسام' 'آستر' 'آسر' 'آسف' 'آسفا' 'آسفين'
 'آسلي' 'آسيا' 'آسيد' 'آسيوي' 'آسيوية' 'آش' 'آشلي' 'آشور' 'آشيل' 'آفات'
 'آفاق' 'آفة' 'آفي' 'آفييشن' 'آكل' 'آكلات' 'آكلة' 'آكلش' 'آكله' 'آكلي']


In [ ]:
df['tf_idf_classifications'] = [get_top_tfidf_words(row) for row in tfidf_matrix]


In [21]:
df.to_csv("./data/tf_idf_classifications.csv", index=False)

In [22]:
df = pd.read_csv("./data/tf_idf_classifications.csv")
df.head()

,file_name,episode_transcript,length,tags,creator,cleaned_transcript,classifications,tf_idf_classifications
0,أطفال_غزة,بعد سبعه اشهر وانا لا اجرؤ على حذف شيء\nيخصك م...,00:00:35,"['غزة', 'الدحيح', 'AJ+ عربي', 'Eldahih السليط'...",Al Mokhbir Al Eqtisadi,سبعه اشهر وانا اجرؤ حذف شيء يخصك ازيل تطبيقات ...,"['امهات غزه', 'اطفال غزه', 'قتل الاطفال', 'رفح...","عينكم, خلوا, قصه"
1,المخبر_الاقتصادي___كيف_أصبح_أصدقاء_إسرائيل_من_...,عمان وقطر والبحرين والسعوديه دي هي الدول\nالعر...,00:22:52,"['اسرائيل', 'اشرف ابراهيم', 'البرلمان البريطان...",Al Mokhbir Al Eqtisadi,عمان وقطر والبحرين والسعوديه الدول العربيه الا...,"['العلاقات البريطانية الإسرائيلية', 'اللوبي ال...","المحافظين, الاسرائيلي, حزب"
2,المخبر_الاقتصادي___كيف_بنى_المهاجرون_المغاربيو...,على غير العادة وبدون أي مقدمات\nخلونا نبدأ الح...,00:20:44,"['الجزائر', 'المغرب العربي', 'المغرب الكبير', ...",Al Mokhbir Al Eqtisadi,العادة وبدون مقدمات خلونا نبدأ الحلقة وندخل ال...,"['الهجرة المغاربية إلى فرنسا', 'الاستعمار الفر...","المغاربة, المغاربية, فرنسا"
3,المخبر_الاقتصادي___كيف_تخطط_أمريكا_لمنع_تطور_ق...,من حوالي السنتين\nوتحديداً في 3 ديسمبر 2020\nج...,00:14:52,"['AMD', 'APPLE', 'CHINA', 'NVDA', 'QCOM', 'SEC...",Al Mokhbir Al Eqtisadi,حوالي السنتين وتحديدا جون راتكليف مدير الاستخب...,"['الحرب التكنولوجية بين أمريكا والصين', 'صناعة...","الصين, دا, أشباه"
4,المخبر_الاقتصادي___كيف_سرقت_بريطانيا_كوه_نور_أ...,يوم 8 سبتمبر 2022\nتم إعلان وفاة ملكة بريطانيا...,00:14:54,"['الملكة', 'الملكة إليزابيث', 'المملكة المتحدة...",Al Mokhbir Al Eqtisadi,إعلان وفاة ملكة بريطانيا إليزبيث عمر يناهز الش...,"['جوهرة كوه نور', 'الاستعمار البريطاني للهند',...","كوه, نور, الماسة"
